### 1. calculate daily weighted return

In [109]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle

startDate = '20200227'
endDate = '20200227'

readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]

def findTmValue(clockLs, tm, method='L', buffer=0):
    maxIx = len(clockLs)
    orignIx = np.arange(maxIx)
    if method == 'F':
        ix = np.searchsorted(clockLs, clockLs+(tm-buffer))
        ## if target future index is next tick, mask
        mask = (orignIx == (ix - 1))|(orignIx == ix)|(ix == maxIx)
    elif method == 'L':
        ## if target future index is last tick, mask
        ix = np.searchsorted(clockLs, clockLs-(tm-buffer))
        ix = ix - 1
        ix[ix<0] = 0
        mask = (orignIx == ix) | ((clockLs-(tm-buffer)).values < clockLs.values[0])
    ix[mask] = -1
    return ix

mdData1 = pd.DataFrame()
mdindex1 = pd.DataFrame()
order1 = pd.DataFrame()

# 1. load mdLog data

for date in dateLs:
    
    readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
    orderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%date))
    orderLog['order'] = orderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog['startClock'] = orderLog.groupby(['order'])['clockAtArrival'].transform('first')
    orderLog['duration'] = orderLog['clockAtArrival'] - orderLog['startClock']
    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
    
    ### make sure no order has shares > 80w or notional > 800w
    if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
        print('some order quantity are > 80w')
        display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection']])
    if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
        print('some order ntl are > 800w')
        display(orderLog[orderLog['absOrderSize'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                         'orderNtl', 'orderDirection']])
    ### make sure same direction in same colo_broker
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        print('opposite direction for same date, same secid, same vai')
        display(orderLog[orderLog['directNum'] != 1][['date', 'accCode', 'secid', 'vai', 'orderDirection']])
        orderLog = orderLog[orderLog['directNum'] == 1]
    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection'].nunique() == 1).all() == True)
    ## make sure each account, secid, vai only has one insertion
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        print('more than one insertion at same time')
        a = a[a>1].reset_index()
        display(a)
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]
    ### make sure there is no unexpected updateType 
    def getTuple(x):
        return tuple(i for i in x)
    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 4, 1, 4)]), 0,
                     np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3)]), 1,
                     np.where(checkLog['updateType'] == (0, 2, 1, 3), 2,
                     np.where(checkLog['updateType'].isin([(0, 3)]), 3,
                     np.where(checkLog['updateType'].isin([(0, ), (0, 2), (0, 2, 1)]), 4, 5)))))
    display(checkLog[checkLog['status'] == 5])
    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)
    ### check status==0 got all traded
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in total trade, any fill != total cases')
    display(a[a['filled'] != a['total']])
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    ### check status==1 got partial traded
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
    display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    ### check if any cancellation within 1 sec
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    print('any cancellation within 1 sec')
    display(a)
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]
    
    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)
    
    li_st = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\orderLog\code\indexCatInfo.csv', encoding="utf-8")
    order = orderLog[orderLog["updateType"] == 0]
    trade = orderLog[orderLog["updateType"] == 4]
    order = pd.merge(order, trade.loc[:, ['order', 'clockAtArrival', 'tradePrice', 'tradeNtl']], on='order', validate="one_to_many")
    order["interval"] = order["clockAtArrival_y"] - order["clockAtArrival_x"]
    order = order.loc[:, ['date', "secid", "vai", "interval", "orderDirection", "tradePrice_y", "isMsg", "tradeNtl_y","sdd","ars", "exchange", "colo", "accCode", "order"]]
    order = order.rename(columns={"secid": "ID", "tradePrice_y": "tradePrice", "tradeNtl_y": "tradeNtl", "date": "Date", "vai": "cum_volume"})
    order = pe.merge(order, li_st, left_on="ID", right_on="secid")
    stock_list = order['ID'].unique()
    
    order1 = pd.concat([order, order1])
    
    
    readPath = r'\\192.168.10.30\Kevin_zhenyu\rawData\logs_%s_zs_92_01_day_data'%date
    mdDataSHPath = glob.glob(os.path.join(readPath, 'mdLog_SH***.csv'))[-1]
    mdDataSH = pd.read_csv(mdDataSHPath)
    mdDataSH['ID'] = mdDataSH['StockID'] + 1000000
    
    mdindex = mdDataSH[mdDataSH['StockID'] < 600000]
    mdindex['time'] = mdindex.time.str.slice(0, 2) + mdindex.time.str.slice(3, 5) + mdindex.time.str.slice(6, 8) + '000'
    mdindex['time'] = mdindex['time'].astype('int64')
    mdindex['time'] = mdindex.groupby(['StockID'])['time'].cummax()
    mdindex['max_cum_volume'] = mdindex.groupby(['StockID'])['cum_volume'].cummax()
    mdindex = mdindex[mdindex["StockID"] != 16]
    mdindex["index1"] = np.where(mdindex["StockID"] == 300, "IF", np.where(mdindex["StockID"] == 905, "IC", "CSI1000"))
    mdindex = mdindex[(mdindex['cum_volume'] > 0) & (mdindex['time'] >= 93000000) &\
                    (mdindex['cum_volume'] == mdindex['max_cum_volume'])]
    mdindex = mdindex[['StockID', 'clockAtArrival', 'sequenceNo', 'time', 'close', "index1"]]
    
    mdDataSH = mdDataSH[mdDataSH["ID"].isin(stock_list)]
    mdDataSH = mdDataSH[mdDataSH['StockID'] >= 600000]
    mdDataSH['time'] = mdDataSH.time.str.slice(0, 2) + mdDataSH.time.str.slice(3, 5) + mdDataSH.time.str.slice(6, 8) + '000'
    mdDataSH['time'] = mdDataSH['time'].astype('int64')
    mdDataSH['time'] = mdDataSH.groupby(['ID'])['time'].cummax()
    mdDataSH['max_cum_volume'] = mdDataSH.groupby(['StockID'])['cum_volume'].cummax()
    mdDataSH = mdDataSH[(mdDataSH['cum_volume'] > 0) & (mdDataSH['time'] >= 93000000) &\
                    (mdDataSH['cum_volume'] == mdDataSH['max_cum_volume'])]
    mdDataSH = mdDataSH[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]
    
    mdDataSZPath = glob.glob(os.path.join(readPath, 'mdLog_SZ***.csv'))[-1]
    mdDataSZ = pd.read_csv(mdDataSZPath)
    mdDataSZ['ID'] = mdDataSZ['StockID'] + 2000000
    mdDataSZ = mdDataSZ[mdDataSZ["ID"].isin(stock_list)]
    
    mdDataSZ['time'] = mdDataSZ.time.str.slice(0, 2) + mdDataSZ.time.str.slice(3, 5) + mdDataSZ.time.str.slice(6, 8) + '000'
    mdDataSZ['time'] = mdDataSZ['time'].astype('int64')
    mdDataSZ['time'] = mdDataSZ.groupby(['ID'])['time'].cummax()
    mdDataSZ['max_cum_volume'] = mdDataSZ.groupby(['StockID'])['cum_volume'].cummax()
    mdDataSZ = mdDataSZ[(mdDataSZ['cum_volume'] > 0) & (mdDataSZ['time'] >= 93000000) &\
                    (mdDataSZ['cum_volume'] == mdDataSZ['max_cum_volume'])]
    mdDataSZ = mdDataSZ[['ID', 'clockAtArrival', 'sequenceNo', 'time', 'cum_volume', 'bid1p', 'ask1p', 'bid1q', 'ask1q', 'bid5q', 'ask5q']]
    
    
    mdData = pd.concat([mdDataSH, mdDataSZ]).reset_index(drop=True)   
    mdData = pd.merge(mdData, li_st, left_on="ID", right_on="secid")
    li_st = li_st[li_st["secid"].isin(stock_list)]
    li_st['index1'] = np.where(li_st["index"] == 'CSIRest', 'CSI1000', li_st["index"])
    list1 = li_st[li_st["index1"] == "IC"]["secid"].values
    list2 = li_st[li_st["index1"] == "IF"]["secid"].values
    list3 = li_st[li_st["index1"] == "CSI1000"]["secid"].values
    
    d1 = mdData[mdData["ID"].isin(list1)]
    d2 = mdindex[mdindex["index1"] == "IC"]
    d1 = pd.concat([d1, d2.loc[:, ["clockAtArrival", "close"]]]).sort_values(by="clockAtArrival")
    d1["close"] = d1["close"].ffill().bfill()
    data = d1[~d1["ID"].isnull()]
    d1 = mdData[mdData["ID"].isin(list2)]
    d2 = mdindex[mdindex["index1"] == "IF"]
    d1 = pd.concat([d1, d2.loc[:, ["clockAtArrival", "close"]]]).sort_values(by="clockAtArrival")
    d1["close"] = d1["close"].ffill().bfill()
    data = pd.concat([data, d1[~d1["ID"].isnull()]])
    d1 = mdData[mdData["ID"].isin(list3)]
    d2 = mdindex[mdindex["index1"] == "CSI1000"]
    d1 = pd.concat([d1, d2.loc[:, ["clockAtArrival", "close"]]]).sort_values(by="clockAtArrival")
    d1["close"] = d1["close"].ffill().bfill()
    data = pd.concat([data, d1[~d1["ID"].isnull()]])
    mdData = data

      
    mdData = mdData.sort_values(by=['ID', 'sequenceNo']).reset_index(drop=True)
    mdData['safeBid1p'] = np.where(mdData['bid1p'] == 0, mdData['ask1p'], mdData['bid1p'])
    mdData['safeAsk1p'] = np.where(mdData['ask1p'] == 0, mdData['bid1p'], mdData['ask1p'])
    mdData['adjMid'] = (mdData['safeBid1p']*mdData['ask1q'] + mdData['safeAsk1p']*mdData['bid1q'])/(mdData['bid1q'] + mdData['ask1q'])
    mdData['session'] = np.where(mdData['time'] >= 130000000, 1, 0)
    mdData = mdData.reset_index(drop=True)
    groupAllData = mdData.groupby(['ID', 'session'])
    mdData['sessionStartCLA'] = groupAllData['clockAtArrival'].transform('min')
    mdData['relativeClock'] = mdData['clockAtArrival'] - mdData['sessionStartCLA']
    mdData['trainFlag'] = np.where(mdData['relativeClock'] > 179.5*1e6, 1, 0)
    mdData['index1'] = mdData.index.values
    mdData['sessionStartIx'] = groupAllData['index1'].transform('min')
    for tm in [90]:
        tmCol = 'F{}s_ix'.format(tm)
        mdData[tmCol] = groupAllData['relativeClock'].transform(lambda x: findTmValue(x, tm*1e6, 'F', 5*1e5)).astype(int)
    for tm in [90]:
        tmIx = mdData['F{}s_ix'.format(tm)].values + mdData['sessionStartIx'].values
        adjMid_tm = mdData['adjMid'].values[tmIx]
        adjMid_tm[mdData['F{}s_ix'.format(tm)].values == -1] = np.nan
        mdData['adjMid_F{}s'.format(tm)] = adjMid_tm
        adjMid_tm1 = mdData['close'].values[tmIx]
        adjMid_tm1[mdData['F{}s_ix'.format(tm)].values == -1] = np.nan
        mdData['close_F{}s'.format(tm)] = adjMid_tm1
    mdData["Date"] = date
    
    del mdDataSH
    del mdDataSZ
    mdData = mdData.loc[:, ['Date', "ID", "clockAtArrival", "cum_volume", "adjMid", "adjMid_F90s",
                            "close", "close_F90s"]]
       
    mdData1 = pd.concat([mdData, mdData1])

    
# # # 2. find each order and trade's forward 90s stock adjMid price
# mdData1["Date"] = mdData1["Date"].astype(int)
# mdData1["isOrder"] == 0
# order1["isOrder"] == 1
# mdData1 = mdData1.rename(columns={"clockAtArrival": "newclockAtArrival"})
# d1 = pd.concat([order1, mdData1.loc[:, ['date', 'secid', 'newclockAtArrival', 'cum_volume', 'isOrder']]]).\
#     sort_values(by=["date", "secid", "cum_volume", "isOrder"]).reset_index(drop=True)
# d1["clockAtArrival"] = d1.groupby(["date", "secid"])["clockAtArrival"].ffill().bfill()
# order1 = order1[order1["isOrder"] == 1]
# order1["clockAtArrival"] = np.where(order1["updateType"] == 0, order1["clockAtArrival"],
#                                        order1["clockAtArrival"] + order1['duration'])

# re = pd.concat([order1, mdData1.loc[:, ['Date',"ID", "clockAtArrival", "cum_volume", "adjMid", "adjMid_F90s"]]])
# re = re.sort_values(by=["Date", "ID", "clockAtArrival"]).reset_index(drop=True)
# re["adjMid_F90s"] = re.groupby(["Date", "ID"])["adjMid_F90s"].bfill()
# re = re[re["isOrder"] == 1]
# re["buyRet"] = np.where(re["orderDirection"] == 1, re["adjMid_F90s"]/re["tradePrice"] - 1, np.nan)
# re["sellRet"] = np.where(re["orderDirection"] == -1, re["tradePrice"]/re["adjMid_F90s"] - 1, np.nan)
# print("re:")
# display(re)

# # 3.  find corresponding index return
# del mdData1
# li_st = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\orderLog\code\indexCatInfo.csv', encoding="utf-8")
# re = pd.merge(re, li_st, left_on="ID", right_on="secid")
# re['index1'] = np.where(re["index"] == 'CSIRest', 'CSI1000', re["index"])
# mdindex1["Date"] = mdindex1["Date"].astype(int)
# re = pd.concat([re, mdindex1.loc[:,["Date", "index1", "clockAtArrival", "close_F90s", "close"]]])
# re = re.sort_values(by=["Date", "index1", "clockAtArrival"]).reset_index(drop="True")
# re["close_F90s"] = re.groupby(["Date", "index1"])["close_F90s"].bfill()
# re["close"] = re.groupby(["Date", "index1"])["close"].ffill()
# re["indexRet"] = re["close_F90s"]/re["close"] - 1
# re1 = re[~re["vai"].isnull()]
# del re
# re1 = re1.sort_values(by=["Date", "ID", "clockAtArrival"]).reset_index(drop="True")
# F1 = open(r"\\192.168.10.30\Kevin_zhenyu\orderLog\code\betaInfo.pkl", 'rb')
# df1 = pickle.load(F1)
# df1["Date"] = df1["date"].apply(lambda x: int(x.strftime("%Y%m%d")))
# re1 = pd.merge(df1.loc[:, ["Date", "ID", "beta_60"]], re1, on=["Date", "ID"], how="inner")
# re1["buyAlpha"] = np.where(re1["orderDirection"] == 1, re1["buyRet"] - re1["indexRet"]*re1["beta_60"], np.nan)
# re1["sellAlpha"] = np.where(re1["orderDirection"] == -1, re1["sellRet"] + re1["indexRet"]*re1["beta_60"], np.nan)
# savePath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\result\90s return'
# re1[re1["Date"] == int(startDate)].to_csv(os.path.join(savePath, 'OrderLog_%s.csv'%int(startDate)), index=False)
# re1[re1["Date"] == int(endDate)].to_csv(os.path.join(savePath, 'OrderLog_%s.csv'%int(endDate)), index=False)
# print("re1:")
# display(re1)

# # 4. generate daily weighted return and alpha
# from IPython.display import display, HTML

# re1["buyNtl"] = np.where(~re1["buyRet"].isnull(), re1["tradeNtl"], np.nan)
# re1["sellNtl"] = np.where(~re1["sellRet"].isnull(), re1["tradeNtl"], np.nan)
# re1["sumbuyNtl"] = re1.groupby(["Date", "exchange", "index", "colo"])["buyNtl"].transform(sum)
# re1["sumsellNtl"] = re1.groupby(["Date", "exchange", "index", "colo"])["sellNtl"].transform(sum)

# re1["sumsellRet"] = re1["tradeNtl"] * re1["sellRet"]
# re1["sumsellRet"] = re1.groupby(["Date", "exchange", "index", "colo"])["sumsellRet"].transform(sum)

# re1["sumbuyAlpha"] = re1["tradeNtl"] * re1["buyAlpha"]
# re1["sumbuyAlpha"] = re1.groupby(["Date", "exchange", "index", "colo"])["sumbuyAlpha"].transform(sum)

# re1["sumsellAlpha"] = re1["tradeNtl"] * re1["sellAlpha"]
# re1["sumsellAlpha"] = re1.groupby(["Date", "exchange", "index", "colo"])["sumsellAlpha"].transform(sum)

# re1["sumbuyRet"] = re1["tradeNtl"] * re1["buyRet"]
# re1["sumbuyRet"] = re1.groupby(["Date", "exchange", "index", "colo"])["sumbuyRet"].transform(sum)


# re1["tnw_buyRet"] = re1["sumbuyRet"] / re1["sumbuyNtl"]
# re1["tnw_sellRet"] = re1["sumsellRet"] / re1["sumsellNtl"]
# re1["tnw_buyAlpha"] = re1["sumbuyAlpha"] / re1["sumbuyNtl"]
# re1["tnw_sellAlpha"] = re1["sumsellAlpha"] / re1["sumsellNtl"]

# save1 = re1[re1["Date"] == int(startDate)].groupby(["Date", "exchange", "index", "colo"])["tnw_buyRet", "tnw_buyAlpha", "tnw_sellRet", "tnw_sellAlpha"].first().reset_index()
# save2 = re1[re1["Date"] == int(endDate)].groupby(["Date", "exchange", "index", "colo"])["tnw_buyRet", "tnw_buyAlpha", "tnw_sellRet", "tnw_sellAlpha"].first().reset_index()
# savePath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\result\90s return'
# save1.to_csv(os.path.join(savePath, '90sReturn_%s.csv'%int(startDate)), index=False)
# save2.to_csv(os.path.join(savePath, '90sReturn_%s.csv'%int(endDate)), index=False)

# for col in ["tnw_buyRet", "tnw_sellRet", "tnw_buyAlpha", "tnw_sellAlpha"]:
#     re1[col] = re1[col].apply(lambda x: '%.4f'%(x))
# HTML(re1.groupby(["Date", "exchange", "index", "colo"])["tnw_buyRet", "tnw_buyAlpha", "tnw_sellRet", "tnw_sellAlpha"].first().to_html())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

more than one insertion at same time


,date,accCode,secid,vai,order,clockAtArrival
0,20200302,5470,2300049,46600,2636,2
1,20200302,5470,2300466,171300,2868,2
2,20200302,5470,2300712,15700,3044,2
3,20200302,9243,2002240,81300,8191,2
4,20200302,9451,2300592,22000,10174,2
5,20200302,9454,2300466,171300,10843,2
6,20200302,9665,1603228,2399507,13717,2


,order,updateType,status
4933,4936,"(0, 4, 2, 1, 3)",5
5182,5185,"(0, 4, 2, 4)",5


in total trade, any fill != total cases


...

,order,filled,total
5926,7142,18,6500


in partial trade, any fill >= total or fill is 0 cases for updateType 4


,order,filled,total


any cancellation within 1 sec


,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,absOrderSizeCumFilled,...,ApplSeqNum,order,orderNtl,exchange,tradeNtl,startClock,duration,isMsg,directNum,status


In [110]:
order1 = order1[order1["sdd"] >= 93300000]
stock_list1 = order1[(order1["accCode"] == 8854) & (order1["ars"] == 21) & (order1["Date"] == int(startDate))]["ID"].unique()
stock_list2 = order1[(order1["accCode"] == 8854) & (order1["ars"] == 21) & (order1["Date"] == int(endDate))]["ID"].unique()
display(stock_list1)
display(stock_list2)
order1 = pd.concat([order1[(order1["Date"] == int(startDate)) & (order1["ID"].isin(stock_list1))], 
                    order1[(order1["Date"] == int(endDate)) & (order1["ID"].isin(stock_list2))]])
# stock_list = order1[(order1["accCode"] == 8854) & (order1["ars"] == 21) & (order1["Date"] == int(startDate))]["ID"].unique()
# order1 = order1[order1["ID"].isin(stock_list)]
order1

array([1600057, 1600206, 1600252, 1600284, 1600389, 1600491, 1600552,
       1600571, 1600650, 1600740, 1600850, 1600864, 1601107, 1601126,
       1601137, 1601567, 1603220, 1603279, 1603298, 1603458, 1603708,
       1603801, 1603896, 1603915, 2000070, 2000546, 2000567, 2000831,
       2000915, 2002099, 2002124, 2002135, 2002139, 2002224, 2002254,
       2002298, 2002458, 2002474, 2002518, 2002522, 2002534, 2002545,
       2002609, 2002657, 2002697, 2002733, 2002737, 2002746, 2002859,
       2300034, 2300036, 2300079, 2300130, 2300209, 2300303, 2300476,
       2300602, 2300674, 2300748], dtype=int64)

array([1600206, 1600211, 1600217, 1600295, 1600323, 1600351, 1600366,
       1600449, 1600483, 1600533, 1600552, 1600618, 1600641, 1600708,
       1600771, 1600775, 1600797, 1600812, 1600966, 1600975, 1601222,
       1601375, 1601636, 1603027, 1603105, 1603421, 1603606, 1603626,
       1603636, 1603869, 1603881, 2000040, 2000049, 2000155, 2000546,
       2000550, 2000561, 2000650, 2000665, 2000733, 2000739, 2000811,
       2000836, 2000875, 2000885, 2000892, 2001896, 2002016, 2002025,
       2002035, 2002079, 2002123, 2002139, 2002148, 2002182, 2002197,
       2002224, 2002235, 2002254, 2002258, 2002261, 2002293, 2002332,
       2002378, 2002393, 2002405, 2002436, 2002474, 2002481, 2002497,
       2002534, 2002538, 2002565, 2002621, 2002631, 2002639, 2002649,
       2002657, 2002663, 2002698, 2002717, 2002733, 2002851, 2002913,
       2300007, 2300011, 2300016, 2300034, 2300085, 2300118, 2300143,
       2300209, 2300215, 2300229, 2300262, 2300287, 2300294, 2300327,
       2300352, 2300

,Date,ID,vai,interval,orderDirection,tradePrice,isMsg,tradeNtl,sdd,ars,exchange,colo,accCode,order
905,20200302,1600057,18063329,259881,-1,4.06,0.0,812.0,145358000,21,SSE,zt,8854,5620
2987,20200302,1600206,8486348,1263916,1,14.03,0.0,12627.0,93803000,11,SSE,zt,5269,119
2988,20200302,1600206,8486348,1638728,1,14.03,0.0,9821.0,93803000,11,SSE,zt,5269,119
2989,20200302,1600206,8486348,1639292,1,14.03,0.0,4209.0,93803000,11,SSE,zt,5269,119
2990,20200302,1600206,8486348,3675766,1,14.03,0.0,221674.0,93803000,11,SSE,zt,5269,119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56461,20200303,2300759,4299165,4764,1,62.40,1.0,18720.0,145222170,11,SZE,zs,9758,17548
56563,20200303,2300771,279200,37066,1,76.24,0.0,7624.0,93803000,21,SZE,zt,8854,5463
56564,20200303,2300771,279400,4213,1,76.24,1.0,22872.0,93804500,11,SZE,zs,9758,17552
56565,20200303,2300771,280200,4296,1,76.24,1.0,38120.0,93809750,11,SZE,zs,9758,17553


In [111]:
# 2. find each order and trade's forward 90s stock adjMid price
mdData1["Date"] = mdData1["Date"].astype(int)
mdData1["isOrder"] = 0
order1["isOrder"] = 1
mdData1 = mdData1.rename(columns={"clockAtArrival": "newclockAtArrival"})
order1 = pd.concat([order1, mdData1.loc[:, ['Date', 'ID', 'newclockAtArrival', 'cum_volume', 'isOrder']]]).\
    sort_values(by=["Date", "ID", "cum_volume", "isOrder"]).reset_index(drop=True)
order1["newclockAtArrival"] = order1.groupby(["Date", "ID"])["newclockAtArrival"].ffill().bfill()
order1 = order1[order1["isOrder"] == 1]
order1["newclockAtArrival"] = order1["newclockAtArrival"] + order1['interval']

re = pd.concat([order1, mdData1.loc[:, ['Date', "ID", "newclockAtArrival", "cum_volume", "adjMid",
                                        "adjMid_F90s", 'isOrder', "close", "close_F90s"]]])
del order1
del mdData1
re = re.sort_values(by=["Date", "ID", "cum_volume", "newclockAtArrival"]).reset_index(drop=True)
re["adjMid_F90s"] = re.groupby(["Date", "ID"])["adjMid_F90s"].bfill()
re["close_F90s"] = re.groupby(["Date", "ID"])["close_F90s"].bfill()
re["close"] = re.groupby(["Date", "ID"])["close"].ffill()
re = re[re["isOrder"] == 1]
re["buyRet"] = np.where(re["orderDirection"] == 1, re["adjMid_F90s"] / re["tradePrice"] - 1, np.nan)
re["sellRet"] = np.where(re["orderDirection"] == -1, re["tradePrice"] / re["adjMid_F90s"] - 1, np.nan)
F1 = open(r"\\192.168.10.30\Kevin_zhenyu\orderLog\code\betaInfo.pkl", 'rb')
df1 = pickle.load(F1)
df1["Date"] = df1["date"].apply(lambda x: int(x.strftime("%Y%m%d")))
re = pd.merge(df1.loc[:, ["Date", "ID", "beta_60"]], re, on=["Date", "ID"], how="inner")
re["buyAlpha"] = np.where(re["orderDirection"] == 1, re["buyRet"] - re["indexRet"]*re["beta_60"], np.nan)
re["sellAlpha"] = np.where(re["orderDirection"] == -1, re["sellRet"] + re["indexRet"]*re1["beta_60"], np.nan)

savePath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\result\90s return'
re[re["Date"] == int(startDate)].to_csv(os.path.join(savePath, 'OrderLog1_%s.csv' % int(startDate)), index=False)
re[re["Date"] == int(endDate)].to_csv(os.path.join(savePath, 'OrderLog1_%s.csv' % int(endDate)), index=False)

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


re:


,Date,ID,accCode,adjMid,adjMid_F90s,ars,clockAtArrival,colo,cum_volume,exchange,interval,isMsg,order,orderDirection,sdd,tradeNtl,tradePrice,vai,buyRet,sellRet
0,20200302,1600057,NaN,4.004268,4.011356,NaN,1583112602767618,NaN,188500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20200302,1600057,NaN,4.007950,4.014845,NaN,1583112605986210,NaN,199948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20200302,1600057,NaN,4.015102,4.014930,NaN,1583112608999843,NaN,237484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20200302,1600057,NaN,4.013422,4.015380,NaN,1583112611986595,NaN,271584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20200302,1600057,NaN,4.011745,4.017781,NaN,1583112614953640,NaN,352324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042687,20200303,2300771,NaN,74.998684,75.050000,NaN,1583218614964178,NaN,3336919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042688,20200303,2300771,NaN,75.258095,75.050000,NaN,1583218617955920,NaN,3337419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042689,20200303,2300771,NaN,75.257000,75.050000,NaN,1583218623607598,NaN,3337519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042690,20200303,2300771,NaN,NaN,75.050000,NaN,1583218626278499,NaN,3337619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



re1:


,Date,ID,beta_60,accCode,adjMid,adjMid_F90s,ars,buyRet,clockAtArrival,colo,...,order,orderDirection,sdd,secid,sellRet,tradeNtl,tradePrice,vai,buyAlpha,sellAlpha
0,20200302,1600057,0.753149,8854.0,NaN,4.059069,21.0,NaN,1583132041312098,zt,...,5620.0,-1.0,145358000.0,1600057.0,0.000229,812.0,4.06,18063329.0,NaN,0.000053
1,20200302,1600206,1.368354,9551.0,NaN,13.960323,11.0,-0.004966,1583113086050295,zt,...,12286.0,1.0,93803000.0,1600206.0,NaN,30866.0,14.03,8486348.0,-0.003326,NaN
2,20200302,1600206,1.368354,9551.0,NaN,13.960323,11.0,-0.004966,1583113086145057,zt,...,12286.0,1.0,93803000.0,1600206.0,NaN,1403.0,14.03,8486348.0,-0.003326,NaN
3,20200302,1600206,1.368354,9551.0,NaN,13.960323,11.0,-0.004966,1583113086145267,zt,...,12286.0,1.0,93803000.0,1600206.0,NaN,14030.0,14.03,8486348.0,-0.003326,NaN
4,20200302,1600206,1.368354,9551.0,NaN,13.960323,11.0,-0.004966,1583113086145472,zt,...,12286.0,1.0,93803000.0,1600206.0,NaN,70150.0,14.03,8486348.0,-0.003326,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,20200303,2300726,1.341571,8854.0,NaN,31.105455,21.0,NaN,1583211840857853,zt,...,5458.0,-1.0,130357000.0,2300726.0,-0.000497,6218.0,31.09,6811933.0,NaN,-0.002195
3146,20200303,2300748,0.973193,8854.0,NaN,33.222162,21.0,0.003085,1583199487050809,zt,...,5459.0,1.0,93803000.0,2300748.0,NaN,3312.0,33.12,846502.0,0.002154,NaN
3147,20200303,2300748,0.973193,8854.0,NaN,33.222162,21.0,0.002177,1583199487051189,zt,...,5459.0,1.0,93803000.0,2300748.0,NaN,3315.0,33.15,846502.0,0.001247,NaN
3148,20200303,2300759,0.616331,8854.0,NaN,62.827368,21.0,NaN,1583215236913704,zt,...,5460.0,-1.0,140033000.0,2300759.0,0.000519,6286.0,62.86,3773265.0,NaN,-0.000191


In [6]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML
startDate = '20200430'
endDate = '20200430'

readPath = "F:\\orderLog\\result\\90s return\\"
dataPathLs = np.array(glob.glob(readPath + 'OrderLog1_***.csv'))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
df = []

for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    data["ars"] = data.groupby(['order'])['ars'].transform('first')
    data["sdd"] = data.groupby(['order'])['sdd'].transform('first')
    data['isMsg'] = np.where(data['updateType'] == 0, np.where(data['mse'] == 100, 1, 0), np.nan)
    data['isMsg'] = data.groupby(['order'])['isMsg'].ffill()
    data = data[data["updateType"] == 4]
    data = data[data["sdd"] >= 93300000]
    data["indexRet"] = data["close_F90s"]/data["close"] - 1
    data["buyAlpha"] = np.where(data["orderDirection"] == 1, data["buyRet"] - data["indexRet"] * data["beta_60"], np.nan)
    data["sellAlpha"] = np.where(data["orderDirection"] == -1, data["sellRet"] + data["indexRet"] * data["beta_60"], np.nan)
    data["pp"] = (data.groupby("order").cumcount()+1)[lambda x: x <= 1]
    data["buyNum"] = np.where((data["orderDirection"] == 1) & (data["pp"]==1), 1, 0)
    data["sellNum"] = np.where((data["orderDirection"] == -1) & (data["pp"]==1), 1, 0)
    data["accCode"] = data["colo"] + data["accCode"].astype(int).astype("str")
    df += [data]
df = pd.concat(df, sort=False)
df["sta"] = np.where(df["ars"] == 21, "statwo", "staone")

df["buyNtl"] = np.where(~df["buyRet"].isnull(), df["tradeNtl"], np.nan)
df["sellNtl"] = np.where(~df["sellRet"].isnull(), df["tradeNtl"], np.nan)
df["sumbuyNtl"] = df.groupby(["exchange", "sta"])["buyNtl"].transform(sum)
df["sumsellNtl"] = df.groupby(["exchange", "sta"])["sellNtl"].transform(sum)
    
df["sumsellRet"] = df["tradeNtl"] * df["sellRet"]
df["sumsellRet"] = df.groupby(["exchange", "sta"])["sumsellRet"].transform(sum)
    
df["sumbuyAlpha"] = df["tradeNtl"] * df["buyAlpha"]
df["sumbuyAlpha"] = df.groupby(["exchange", "sta"])["sumbuyAlpha"].transform(sum)
    
df["sumsellAlpha"] = df["tradeNtl"] * df["sellAlpha"]
df["sumsellAlpha"] = df.groupby(["exchange", "sta"])["sumsellAlpha"].transform(sum)
    
df["sumbuyRet"] = df["tradeNtl"] * df["buyRet"]
df["sumbuyRet"] = df.groupby(["exchange", "sta"])["sumbuyRet"].transform(sum)

df["buyRet"] = df["sumbuyRet"] / df["sumbuyNtl"]
df["sellRet"] = df["sumsellRet"] / df["sumsellNtl"]
df["buyAlpha"] = df["sumbuyAlpha"] / df["sumbuyNtl"]
df["sellAlpha"] = df["sumsellAlpha"] / df["sumsellNtl"]
df["buyOrderNum"] = df.groupby(["exchange", "sta"])["buyNum"].transform(sum)
df["sellOrderNum"] = df.groupby(["exchange", "sta"])["sellNum"].transform(sum)

from IPython.display import display, HTML
for col in ["buyRet", "sellRet", "buyAlpha", "sellAlpha"]:
    df[col] = df[col].apply(lambda x: '%.2f'%(x*10000))
        
df.groupby(["exchange", "sta"])["buyOrderNum", "buyRet", "buyAlpha", "sellOrderNum", "sellRet", "sellAlpha"].first().reset_index()

,exchange,sta,buyOrderNum,buyRet,buyAlpha,sellOrderNum,sellRet,sellAlpha
0,SSE,staone,2597,23.46,18.08,4259,4.22,6.97
1,SSE,statwo,69,32.06,22.84,56,-13.55,-8.04
2,SZE,staone,2960,22.72,15.23,4616,7.36,11.63


In [1]:
# 4. generate daily weighted return and alpha
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML
startDate = '20200430'
endDate = '20200430'

readPath = "F:\\orderLog\\result\\90s return\\"
dataPathLs = np.array(glob.glob(readPath + 'OrderLog1_***.csv'))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
df = []

for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    data["ars"] = data.groupby(['order'])['ars'].transform('first')
    data["sdd"] = data.groupby(['order'])['sdd'].transform('first')
    data['isMsg'] = np.where(data['updateType'] == 0, np.where(data['mse'] == 100, 1, 0), np.nan)
    data['isMsg'] = data.groupby(['order'])['isMsg'].ffill()
    data = data[data["updateType"] == 4]
    stock_list = data[(data["accCode"] == 8854) & (data["ars"] == 21) & (data["exchange"] == "SSE")]["secid"].unique()
    display(stock_list)
    data = data[data["secid"].isin(stock_list)]
    data = data[data["sdd"] >= 93300000]
    data["indexRet"] = data["close_F90s"]/data["close"] - 1
    data["buyAlpha"] = np.where(data["orderDirection"] == 1, data["buyRet"] - data["indexRet"] * data["beta_60"], np.nan)
    data["sellAlpha"] = np.where(data["orderDirection"] == -1, data["sellRet"] + data["indexRet"] * data["beta_60"], np.nan)
    data["pp"] = (data.groupby("order").cumcount()+1)[lambda x: x <= 1]
    data["buyNum"] = np.where((data["orderDirection"] == 1) & (data["pp"]==1), 1, 0)
    data["sellNum"] = np.where((data["orderDirection"] == -1) & (data["pp"]==1), 1, 0)
    data["accCode"] = data["colo"] + data["accCode"].astype(int).astype("str")
    df += [data]
df = pd.concat(df, sort=False)
display(df["date"].unique())

df["sta"] = np.where(df["ars"] == 21, "statwo", "staone")

p1 = pd.DataFrame()

for j in df[df["accCode"] != "zt8854"]["accCode"].unique():
    print(j)
    h = df[df["accCode"] == j].groupby("date")["ID"].unique().reset_index()
    df2 = df[df["accCode"] == j]
    df1 = pd.DataFrame()
    for i in range(len(h)):
        dff = df[(df["accCode"] == "zt8854") & (df["date"] == h["date"].iloc[i]) & (df["ID"].isin(h["ID"].iloc[i]))]
        df1 = pd.concat([dff, df1])
    re1 = pd.concat([df1, df2])
    
    re1["buyNtl"] = np.where(~re1["buyRet"].isnull(), re1["tradeNtl"], np.nan)
    re1["sellNtl"] = np.where(~re1["sellRet"].isnull(), re1["tradeNtl"], np.nan)
    re1["sumbuyNtl"] = re1.groupby(["exchange", "accCode", "sta"])["buyNtl"].transform(sum)
    re1["sumsellNtl"] = re1.groupby(["exchange", "accCode", "sta"])["sellNtl"].transform(sum)
    
    re1["sumsellRet"] = re1["tradeNtl"] * re1["sellRet"]
    re1["sumsellRet"] = re1.groupby(["exchange", "accCode", "sta"])["sumsellRet"].transform(sum)
    
    re1["sumbuyAlpha"] = re1["tradeNtl"] * re1["buyAlpha"]
    re1["sumbuyAlpha"] = re1.groupby(["exchange", "accCode", "sta"])["sumbuyAlpha"].transform(sum)
    
    re1["sumsellAlpha"] = re1["tradeNtl"] * re1["sellAlpha"]
    re1["sumsellAlpha"] = re1.groupby(["exchange", "accCode", "sta"])["sumsellAlpha"].transform(sum)
    
    re1["sumbuyRet"] = re1["tradeNtl"] * re1["buyRet"]
    re1["sumbuyRet"] = re1.groupby(["exchange", "accCode", "sta"])["sumbuyRet"].transform(sum)

    re1["buyRet"] = re1["sumbuyRet"] / re1["sumbuyNtl"]
    re1["sellRet"] = re1["sumsellRet"] / re1["sumsellNtl"]
    re1["buyAlpha"] = re1["sumbuyAlpha"] / re1["sumbuyNtl"]
    re1["sellAlpha"] = re1["sumsellAlpha"] / re1["sumsellNtl"]
    re1["buyOrderNum"] = re1.groupby(["exchange", "accCode", "sta"])["buyNum"].transform(sum)
    re1["sellOrderNum"] = re1.groupby(["exchange", "accCode", "sta"])["sellNum"].transform(sum)

    from IPython.display import display, HTML
    for col in ["buyRet", "sellRet", "buyAlpha", "sellAlpha"]:
        re1[col] = re1[col].apply(lambda x: '%.2f'%(x*10000))
        
    test = re1.groupby(["exchange", "accCode", "sta"])["buyOrderNum", "buyRet", "buyAlpha", "sellOrderNum", "sellRet", "sellAlpha"].first().reset_index()
    p = pd.merge(test[test["accCode"] != "zt8854"], test[test["accCode"] == "zt8854"], on="exchange")
    print(p)
    p1 = pd.concat([p, p1])

HTML(p1.groupby(["exchange", "accCode_x", "accCode_y"])["buyOrderNum_x", "buyOrderNum_y", "buyRet_x", "buyRet_y", "buyAlpha_x", 
                                                  "buyAlpha_y", "sellOrderNum_x", "sellOrderNum_y", "sellRet_x", "sellRet_y",
                                                 "sellAlpha_x", "sellAlpha_y"].first().to_html())


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array([1600217., 1600491., 1600684., 1600797., 1600810., 1600825.,
       1601567., 1603308., 1603557., 1603612., 1603676., 1603718.,
       1603733.])

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (30,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array([1600075., 1600076., 1600110., 1600114., 1600123., 1600129.,
       1600242., 1600284., 1600285., 1600363., 1600449., 1600456.,
       1600459., 1600467., 1600496., 1600502., 1600508., 1600517.,
       1600562., 1600571., 1600668., 1600685., 1600710., 1600715.,
       1600740., 1600783., 1600800., 1600850., 1600855., 1600864.,
       1600971., 1600986., 1600987., 1601058., 1601116., 1601137.,
       1601366., 1601595., 1601666., 1601677., 1601700., 1601886.,
       1601918., 1603127., 1603128., 1603318., 1603323., 1603363.,
       1603612., 1603618., 1603636., 1603686., 1603718., 1603733.,
       1603773., 1603777., 1603803., 1603869., 1603900., 1603915.,
       1603920., 1603956., 1603979.])

array([1600093., 1600559., 1600568., 1600590., 1600596., 1600668.,
       1600710., 1600783., 1600797., 1600810., 1600850., 1600855.,
       1600864., 1600868., 1601069., 1601375., 1603013., 1603421.,
       1603618., 1603676., 1603693., 1603708., 1603711., 1603808.,
       1603915.])

...

array([1600278., 1600305., 1600517., 1600668., 1600681., 1600850.,
       1600975., 1601519., 1603013., 1603018., 1603105., 1603113.,
       1603327., 1603357., 1603393., 1603606., 1603648., 1603680.,
       1603773., 1603898.])

array([1600057., 1600206., 1600252., 1600284., 1600389., 1600491.,
       1600552., 1600571., 1600650., 1600740., 1600850., 1600864.,
       1601107., 1601126., 1601137., 1601567., 1603220., 1603279.,
       1603298., 1603458., 1603708., 1603801., 1603896., 1603915.])

array([1600206., 1600211., 1600217., 1600295., 1600323., 1600351.,
       1600366., 1600449., 1600483., 1600533., 1600552., 1600618.,
       1600641., 1600708., 1600771., 1600775., 1600797., 1600812.,
       1600966., 1600975., 1601222., 1601375., 1601636., 1603027.,
       1603105., 1603421., 1603606., 1603626., 1603636., 1603869.,
       1603881.])

array([20200224, 20200225, 20200226, 20200227, 20200228, 20200302,
       20200303], dtype=int64)

,date,indexCat,ID,beta_60,ApplSeqNum,aaa,absFilledThisUpdate,absOrderSize,absOrderSizeCumFilled,accCode,...,buyRet,sellRet,isMsg,indexRet,buyAlpha,sellAlpha,pp,buyNum,sellNum,sta


In [52]:
hehe = df[(df["accCode"] == "zt5269")]
hehe["order1"] = hehe.groupby(['date', 'ID', 'cum_volume']).grouper.group_info[0]
len(hehe[hehe["orderDirection"] == -1]["order1"].unique())

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


633

In [49]:
df[(df["accCode"] == "zt5269") & (df["order"] == 114)].loc[:, ["date", "ID", "absFilledThisUpdate", "accCode", "orderDirection", "updateType", "cum_volume"]]

,date,ID,absFilledThisUpdate,accCode,orderDirection,updateType,cum_volume
13796,20200224,1600217,11601.0,zt5269,1.0,4.0,24913560.0
13797,20200224,1600217,10000.0,zt5269,1.0,4.0,24913560.0
13798,20200224,1600217,500.0,zt5269,1.0,4.0,24913560.0
13799,20200224,1600217,23000.0,zt5269,1.0,4.0,24913560.0
13800,20200224,1600217,3000.0,zt5269,1.0,4.0,24913560.0
13801,20200224,1600217,1500.0,zt5269,1.0,4.0,24913560.0
13802,20200224,1600217,3000.0,zt5269,1.0,4.0,24913560.0
13803,20200224,1600217,1000.0,zt5269,1.0,4.0,24913560.0
13804,20200224,1600217,2000.0,zt5269,1.0,4.0,24913560.0
13805,20200224,1600217,5000.0,zt5269,1.0,4.0,24913560.0


In [28]:
h = re1[re1["accCode"] == "zt5269"].groupby("date")["ID"].unique().reset_index()
df2 = re1[re1["accCode"] == "zt5269"]
df1 = pd.DataFrame()
for i in range(len(h)):
    df = re1[(re1["accCode"] == "zt8854") & (re1["date"] == h["date"].iloc[i]) & (re1["ID"].isin(h["ID"].iloc[i]))]
    df1 = pd.concat([df, df1])



set()

set()

In [15]:
test = re1.groupby(["exchange", "accCode", "sta"])["buyOrderNum", "buyRet", "buyAlpha", "sellOrderNum", "sellRet", "sellAlpha"].first().reset_index()
p = pd.merge(test[test["accCode"] != "zt8854"], test[test["accCode"] == "zt8854"], on="exchange")
HTML(p.groupby(["exchange", "accCode_x", "accCode_y"])["buyOrderNum_x", "buyOrderNum_y", "buyRet_x", "buyRet_y", "buyAlpha_x", 
                                                  "buyAlpha_y", "sellOrderNum_x", "sellOrderNum_y", "sellRet_x", "sellRet_y",
                                                 "sellAlpha_x", "sellAlpha_y"].first().to_html())

In [35]:
df[df['sta1_x'] == 1]['accCode'].unique()

array(['zs_94_059451', 'zt_52_05537403', 'zt_96_09965801',
       'zt_96_09968501', 'zt_70_017079', 'zs_96_089754', 'zs_96_086282'],
      dtype=object)

### 2. Compare result bi-weekly

In [4]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import pickle
from IPython.display import display, HTML

startDate = '20200113'
endDate = '20200227'

readPath = r'F:\orderLog\result\90s return'
dataPathLs = np.array(glob.glob(os.path.join(readPath, '90sReturn_***.csv')))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
rawReturnLog = []
for thisDate, thisPath in zip(dateLs, dataPathLs):
    data = pd.read_csv(thisPath)
    
    rawReturnLog += [data]
rawReturnLog = pd.concat(rawReturnLog, sort=False)

# 1. index_colo combination with highest Ret, Alpha each day
display(rawReturnLog["Date"].unique())
rawReturnLog["week"] = np.where((rawReturnLog["Date"] >= 20200217) & (rawReturnLog["Date"] <= 20200227), "0217_0227", np.where(
(rawReturnLog["Date"] >= 20200203) & (rawReturnLog["Date"] <= 20200214), "0203_0214", "0113_0123"))
display(HTML(rawReturnLog.groupby(["week", "Date"]).first().to_html()))
# for col in ["tnw_buyRet", "tnw_sellRet", "tnw_buyAlpha", "tnw_sellAlpha"]:
#     rawReturnLog[col] = rawReturnLog[col].apply(lambda x: '%.2f'%(x*10000))
fr = rawReturnLog.groupby(["exchange", "index", "colo", "week"])["tnw_buyRet", "tnw_buyAlpha", "tnw_sellRet", "tnw_sellAlpha"].mean().reset_index()
for col in ["tnw_buyRet", "tnw_sellRet", "tnw_buyAlpha", "tnw_sellAlpha"]:
    fr[col] = fr[col].apply(lambda x: '%.2f'%(x*10000))
HTML(fr.groupby(["exchange", "index", "colo", "week"])["tnw_buyRet", "tnw_buyAlpha", "tnw_sellRet", "tnw_sellAlpha"].first().to_html())

array([20200113, 20200114, 20200115, 20200116, 20200117, 20200120,
       20200121, 20200122, 20200123, 20200203, 20200204, 20200205,
       20200206, 20200207, 20200214, 20200217, 20200218, 20200219,
       20200220, 20200221, 20200224, 20200225, 20200226, 20200227],
      dtype=int64)

In [ ]:
df[df['beta_60'].isnull()].groupby('date')['vai'].size()

In [375]:
df1 = pd.read_csv(r'\\192.168.10.30\Kevin_zhenyu\orderLog\result\90s return\OrderLog1_20200227.csv', encoding="utf-8")
df1["updateType"] = df1["updateType"].astype(int)
df1["accCode"] = df1["accCode"].astype(int)
df1["secid"] = df1["secid"].astype(int)
df1 = df1.rename(columns = {"cum_volume": "vai"})
df1["vai"] = df1["vai"].astype(int)
df1["sdd"] = df1.groupby(['order'])['sdd'].transform('first')
df1 = df1[df1["updateType"]==4]

df1.shape[0]

53333

In [376]:
F1 = open(r'\\192.168.10.30\Kevin_zhenyu\orderLog\result\orderLogWithRet_20200227_addAlpha.pkl', 'rb')
df2 = pickle.load(F1)
df2["sdd"] = df2.groupby(['order'])['sdd'].transform('first')
df2 = df2[df2["updateType"]==4]


df1["close"] = df1["close"].round(3)
df2["indexClose"] = df2["indexClose"].round(3)
df1["adjMid_F90s"] = df1["adjMid_F90s"].round(6)
df2["adjMid_F90s"] = df2["adjMid_F90s"].round(6)
df1["close_F90s"] = df1["close_F90s"].round(4)
df2["indexClose_F90s"] = df2["indexClose_F90s"].round(4)

df2.shape[0]

53333

In [389]:
re = pd.merge(df2, df1.loc[:, ['date', 'accCode', "order", 'secid', 'vai',"clockAtArrival", "tradeNtl", "close_F90s", "close", "indexRet", "adjMid_F90s"]], on=['date', 'accCode', 'secid', 'vai', "clockAtArrival", "tradeNtl"], how="inner")
h = re[(re["close_F90s"] != re["indexClose_F90s"]) | (re["close"] != re["indexClose"]) | (re["adjMid_F90s_x"] != re["adjMid_F90s_y"])].groupby(['date', 'accCode', 'secid', 'vai'])
display(re[(re["close_F90s"] != re["indexClose_F90s"])].loc[:, ['sdd', 'accCode', 'secid', 'vai', "order_x", "order_y", "updateType", "tradeNtl","close_F90s","indexClose_F90s"]])
display(re[(re["close"] != re["indexClose"])].loc[:, ['sdd', 'accCode', 'secid', 'vai', "order_x", "order_y", "updateType", "tradeNtl","close","indexClose", "close_F90s","indexClose_F90s", "vai"]])
display(re[(re["adjMid_F90s_x"] != re["adjMid_F90s_y"])].loc[:, ['sdd', 'accCode', 'secid', 'vai', "order_x", "order_y", "updateType", "tradeNtl","adjMid_F90s_x","adjMid_F90s_y"]])
hehe = re[(re["adjMid_F90s_x"] != re["adjMid_F90s_y"]) | (re["close"] != re["indexClose"]) | (re["close_F90s"] != re["indexClose_F90s"])]\
        .loc[:, ['sdd', 'accCode', 'secid', 'vai', "order_x", "order_y", "updateType", "tradeNtl","adjMid_F90s_y","adjMid_F90s_x", "close_F90s","indexClose_F90s", "close","indexClose"]]
hehe.groupby("secid")["close"].count().reset_index().sort_values(by="close", ascending=False)
# display(hehe)

,sdd,accCode,secid,vai,order_x,order_y,updateType,tradeNtl,close_F90s,indexClose_F90s
131,145124000,9665,1600017,13037917,14777,14777.0,4,11438.0,5751.1994,5750.8874
132,145124000,9665,1600017,13037917,14777,14777.0,4,26600.0,5751.1994,5750.8874
133,145124000,9665,1600017,13037917,14777,14777.0,4,3458.0,5751.1994,5750.8874
134,145124000,9665,1600017,13037917,14777,14777.0,4,46550.0,5751.1994,5750.8874
135,145124000,9665,1600017,13037917,14777,14777.0,4,1596.0,5751.1994,5750.8874
...,...,...,...,...,...,...,...,...,...,...
51714,93000090,9471,2300771,6400,13438,13438.0,4,7460.0,6279.1862,6276.2283
51715,93000090,9471,2300771,6400,13438,13438.0,4,44760.0,6279.1862,6276.2283
51730,93845830,9243,2300775,351900,8899,8899.0,4,3240.0,6231.3327,6232.0591
51744,93855860,9471,2300776,172100,13450,13450.0,4,26800.0,6233.6753,6232.9163


,sdd,accCode,secid,vai,order_x,order_y,updateType,tradeNtl,close,indexClose,close_F90s,indexClose_F90s,vai
295,93038000,9441,1600037,1353500,9406,9406.0,4,2100.0,5796.012,5797.070,5785.6941,5785.6941,1353500
407,95949000,9551,1600054,2253964,13460,13460.0,4,5142.0,6211.818,6211.086,6193.5812,6192.8645,2253964
568,130329000,5269,1600067,3447400,9,9.0,4,3080.0,6301.586,6300.920,6297.5051,6297.0084,3447400
585,135329000,9667,1600068,100831083,15920,15920.0,4,61287.0,4094.687,4094.725,4095.5530,4095.5530,100831083
640,145614000,9741,1600070,21448678,16414,16414.0,4,85824.0,6229.233,6229.402,6232.7942,6232.7942,21448678
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50260,93006130,5470,2300651,20160,3154,3154.0,4,32786.0,6262.948,6270.023,6275.3507,6275.3507,20160
50276,93006130,9454,2300651,20160,11949,11949.0,4,15132.0,6262.948,6270.023,6275.3507,6275.3507,20160
50422,93920700,5470,2300668,194600,3166,3166.0,4,30690.0,6218.047,6218.814,6236.4739,6236.4739,194600
50800,95900810,5470,2300693,707000,3212,3212.0,4,6928.0,6222.002,6223.194,6200.9300,6200.9300,707000


,sdd,accCode,secid,vai,order_x,order_y,updateType,tradeNtl,adjMid_F90s_x,adjMid_F90s_y
9,140746000,9667,1600000,23894647,15873,15873.0,4,393888.0,11.197613,11.197634
120,140716000,9551,1600012,2305210,13457,13457.0,4,2136.0,5.339022,5.339024
121,140716000,9658,1600012,2305210,14306,14306.0,4,534.0,5.339022,5.339024
122,140716000,9658,1600012,2305210,14306,14306.0,4,1602.0,5.339022,5.339024
131,145124000,9665,1600017,13037917,14777,14777.0,4,11438.0,2.656157,2.656076
...,...,...,...,...,...,...,...,...,...,...
51714,93000090,9471,2300771,6400,13438,13438.0,4,7460.0,74.688571,74.670000
51715,93000090,9471,2300771,6400,13438,13438.0,4,44760.0,74.688571,74.670000
51728,93845830,9243,2300775,351900,8899,8899.0,4,25920.0,32.511765,32.511603
51744,93855860,9471,2300776,172100,13450,13450.0,4,26800.0,134.386667,134.461429


,secid,close
34,1600143,170
628,2300059,151
119,1600512,109
54,1600206,90
706,2300370,56
...,...,...
441,1603908,1
439,1603900,1
437,1603895,1
436,1603888,1


In [396]:
hehe[hehe["secid"] == 2300059].loc[:, ["sdd", 'accCode', 'secid', 'vai', "order_x", "order_y", "tradeNtl","close","indexClose", "close_F90s","indexClose_F90s","adjMid_F90s_y", "adjMid_F90s_x"]]

,sdd,accCode,secid,vai,order_x,order_y,tradeNtl,close,indexClose,close_F90s,indexClose_F90s,adjMid_F90s_y,adjMid_F90s_x
42763,93000000,9461,2300059,2910400,12273,12273.0,1516800.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42764,93000000,9765,2300059,2910400,20698,20698.0,11376.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42765,93000000,9765,2300059,2910400,20698,20698.0,37920.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42766,93000000,9765,2300059,2910400,20698,20698.0,30336.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42767,93000000,9765,2300059,2910400,20698,20698.0,56880.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42909,93000670,8867,2300059,4862672,7825,7825.0,9485.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42910,93000670,8867,2300059,4862672,7825,7825.0,3794.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42911,93000670,8867,2300059,4862672,7825,7825.0,17073.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
42912,93000670,8867,2300059,4862672,7825,7825.0,18970.0,4084.078,4084.078,4090.9744,4090.5663,18.782164,18.790004
